In [1]:
import pandas as pd
import pypsa
import numpy as np
network = pypsa.Network()

In [30]:
bus = pd.read_csv('data/base_grid/bus.csv')
sub = pd.read_csv('data/base_grid/sub.csv')
sub = sub.drop(columns = 'interconnect')
bus2sub = pd.read_csv('data/base_grid/bus2sub.csv')
bus2sub = bus2sub.drop(columns = 'interconnect')

In [31]:
buses = bus.merge(bus2sub, how = 'left', on='bus_id')
buses = buses.merge(sub,how='left',on='sub_id')

In [32]:
buses['Va'] = buses['Va'].apply(np.deg2rad)
buses['control'] = buses['type'].map({1:'PQ',2:'PV'})
buses.drop(inplace=True,columns = ['type'])
buses.dropna(subset=['control'],inplace=True)

In [33]:
buses = buses.set_index('bus_id')
buses_columns = {'lat':'y',
                                'lon':'x',
                                'Vm':'v_mag_pu',
                                'Pd':'p',
                                'Qd':'q',
                                'Va':'v_ang',
                                'baseKV':'v_nom',
                                'Vmax':'v_mag_pu_max',
                                'Vmin':'v_mag_pu_min'}

buses = buses.rename(columns = buses_columns)
buses = buses[buses_columns.values()]

In [34]:
buses

,y,x,v_mag_pu,p,q,v_ang,v_nom,v_mag_pu_max,v_mag_pu_min
bus_id,,,,,,,,,
1,43.9761,-70.2211,1.03511,0.00,0.00,-1.124168,230.0,1.1,0.9
2,43.9761,-70.2211,1.03706,0.00,0.00,-1.139032,115.0,1.1,0.9
3,43.9761,-70.2211,1.03293,9.97,1.30,-1.165906,18.0,1.1,0.9
4,43.9761,-70.2211,1.03744,0.00,0.00,-1.140197,1.0,1.1,0.9
5,43.9761,-70.2211,1.03607,0.00,0.00,-1.124050,230.0,1.1,0.9
...,...,...,...,...,...,...,...,...,...
3008156,31.0919,-96.6950,1.02611,0.00,0.00,-0.804766,230.0,1.1,0.9
3008157,31.0919,-96.6950,1.02808,0.00,0.00,-0.872862,115.0,1.1,0.9
3008158,30.7217,-96.4608,1.02650,0.00,0.00,-0.688833,500.0,1.1,0.9


In [35]:
network.import_components_from_dataframe(buses,"Bus")

ERROR:pypsa.io:Error, new components for Bus are not unique


In [49]:
generators = pd.read_csv('data/base_grid/plant.csv')
cost = pd.read_csv('data/base_grid/gencost.csv')
hydro = pd.read_csv('data/base_grid/hydro.csv')
solar = pd.read_csv('data/base_grid/solar.csv')
wind = pd.read_csv('data/base_grid/wind.csv')
generators = generators.merge(cost.drop(columns = ['type','interconnect']),how='left',on='plant_id')
generators = generators.merge(bus.drop(columns = ['type', 'interconnect']),how='left',on='bus_id')


In [50]:
generators

In [86]:
outputs = solar.merge(hydro).merge(wind)

In [95]:
def output(row):
    try:
        return outputs[str(row['plant_id'])]
    except Exception as e:
        return np.nan

In [97]:
generators['output'] = generators.apply(lambda x: output(x), axis=1)

In [110]:
generators[generators.output.isna()]

,plant_id,bus_id,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,...,Va,baseKV,loss_zone,Vmax,Vmin,lam_P,lam_Q,mu_Vmax,mu_Vmin,output
0,0,845,186.58,40.25,95.08,-20.73,1.0090,224.16,1,177.194,...,-58.1969,13.8,1,1.1,0.9,63.95,0,0,0,NaN
1,1,850,211.25,32.45,98.56,-25.80,1.0488,253.80,1,206.883,...,-60.4536,18.0,1,1.1,0.9,63.95,0,0,0,NaN
2,2,851,211.25,53.96,98.56,-25.80,1.0500,253.80,1,206.883,...,-60.7644,13.8,1,1.1,0.9,63.95,0,0,0,NaN
3,3,852,211.25,53.57,98.56,-25.80,1.0545,253.80,1,206.883,...,-59.7440,13.8,1,1.1,0.9,63.95,0,0,0,NaN
4,4,853,211.25,32.45,98.56,-25.80,1.0488,253.80,1,206.883,...,-60.4325,13.8,1,1.1,0.9,63.95,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13946,13946,3005318,0.00,0.00,0.00,0.00,1.0000,500.00,1,460.000,...,-57.6485,161.0,18,1.1,0.9,0.00,0,0,0,NaN
13947,13947,3007305,0.00,0.00,0.00,0.00,1.0000,400.00,1,350.000,...,-28.9509,230.0,1,1.1,0.9,0.00,0,0,0,NaN
13948,13948,3007305,0.00,0.00,0.00,0.00,1.0000,400.00,1,350.000,...,-28.9509,230.0,1,1.1,0.9,0.00,0,0,0,NaN
13949,13949,3005318,0.00,0.00,0.00,0.00,1.0000,400.00,1,335.000,...,-57.6485,161.0,18,1.1,0.9,0.00,0,0,0,NaN


In [111]:
generators.shape

(14050, 54)

In [116]:
solar.shape

(8784, 1140)

In [114]:
5267+8784

14051

In [117]:
hydro.shape

(8784, 3044)

In [127]:
generators.plant_id.nunique()

14050

In [135]:
generators[generators.output.isna()].type.value_counts()

ng            4878
dfo           2509
coal          1119
other          140
nuclear         96
geothermal      42
Name: type, dtype: int64